# Wildcard Handling with Astroquery.mast

----

## Learning Goals

By the end of this tutorial, you will:

* Learn about the wildcards available for `astroquery.mast.Observations` criteria queries
* Learn how to use these wildcards to broaden or refine `astroquery.mast.Observations` criteria queries
* Learn the downfalls of the `instrument_name` criteria, particularly for JWST queries
* Learn how to query for moving targets using target ephemeris and time criteria such as `t_min` and `t_max` for more advanced searches

## Introduction

This notebook will demonstrate the use of wildcards in `astroquery.mast.Observations` criteria queries. The use of wildcards is encouraged for certain criteria types (`string` object types) to ensure that your query results capture what is truly available on MAST. We will use 3 examples to demonstrate some use-cases for wildcards when doing criteria queries, and to emphasize on certain criteria where wildcard usage is highly encouraged, particularly for JWST queries. We will also use the last example to demonstrate the use of value ranges when working with `float` object criteria types.

The workflow for this notebook consists of:

* Wildcards with `astroquery.mast.Observations`
    * Wildcard Search with `instrument_name`
    * Wildcard Search with `instrument_name` and `proposal_id`
    * Wildcard Search a Time-sensitive Object with `target_name` and `t_min`
    * Resources

## Imports

In [ ]:
import astropy.units as u
import matplotlib.pyplot as plt

from astropy.coordinates import SkyCoord
from astropy.table import Table, unique, vstack
from astropy.time import Time
from astroquery.mast import Observations

----

## Wildcards with `astroquery.mast.Observations`

The use of wildcards when making `astroquery.mast.Observations` queries can help ensure you retrieve all observations without leaving anything out. The available wildcards are `%` and `*` with `%` serving to replace a single character, and `*` serving to replace more than one character preceding, following, or in between the existing characters, depending on its placement. See the [Observation Criteria Queries](https://astroquery.readthedocs.io/en/latest/mast/mast.html#observation-criteria-queries) section in the `astroquery.mast` documentation for more information on the wildcards.

With that being said, wildcards are only available for certain criteria. Those criteria that are `string` type objects are criteria that accept wildcards, but `float`, `integer`, or any other objects do not accept wildcards.

Users may call the `get_metadata` method to see the list of criteria that can be queried with. The criteria listed as `string` objects under the **Data Type** column are criteria that can be called with wildcards:

In [ ]:
Observations.get_metadata("observations").show_in_notebook()

### Wildcard Search with `instrument_name`

For our first example we will search for all NIRISS observations taken by a certain proposal/program PI. For this case, our two criteria we query with are `proposal_pi` and `instrument_name`, which are both `string` object criteria. As such, both can be wildcarded for ease of use. In fact, there may be cases where it is necessary to wildcard the `instrument_name` variable. This is because the JWST science instrument names on MAST have been modified to include configuration information for more precise advanced searches. However, this may lead to an incomplete list of observations returned for those who are not trying to do an advanced search. 

We will demonstrate this by looking at the results for the query below:

In [ ]:
observations = Observations.query_criteria(proposal_pi="Espinoza, Nestor",
                                           instrument_name="*NIRISS*")
observations

Here we have a list of all kinda of NIRISS observations led by the PI Dr. Espinoza. Let's call for all the unique values under the `instrument_name` column to see what was returned thanks to our `*` wildcards.

In [ ]:
set(observations['instrument_name'])

As you can see, there were some observations with `instrument_name` labeled something other than `NIRISS`, due to the new configuration on MAST. Had we only searched for the `instrument_name` value `NIRISS`, we would have missed several observations. For the full list of instrument name variations, you can refer to [this page](https://outerspace.stsci.edu/display/MASTDOCS/JWST+Instrument+Names).

**Note**: Be aware that you can be too generous with the wildcards. Too much ambiguity will lead to getting results that were not intended. See below for example.

In [ ]:
observations = Observations.query_criteria(proposal_pi='Espinoza, Nestor',
                                           instrument_name='*NI*S*')
set(observations['instrument_name'])

This query returns `NIRSPEC/SLIT` observations in addition to the NIRISS ones, which is not what was intended.

### Wildcard Search with `instrument_name` and `proposal_id`

Let's repeat the previous example, but add another `string` criteria and wildcard into the mix. Let's query for all observations belonging to proposal/program IDs that begin with the digits `15`, and see what gets returned:

In [ ]:
observations = Observations.query_criteria(proposal_pi='Espinoza, Nestor',
                                           instrument_name='*NIRISS*',
                                           proposal_id=['15%%'])
set(observations['proposal_id']), set(observations['instrument_name'])

### Create a Moving Target Ephemeris using MAST Observations with Wildcard Search

We will be querying for image observations of Comet 67P Churyumov-Gerasimenko observed through the Hubble Space Telescope's Advanced Camera for Surveys (ACS) Wide Field Camera (WFC). This comet's name can be listed in different ways, so we will use `*` wildcards in our criteria query. For this case, we won't use any wildcards in the `instrument_name` because we don't run into many variations of HST instrument names the way we do for JWST.

In [ ]:
observations = Observations.query_criteria(target_name="*COM*67P*",
                                           instrument_name="ACS/WFC")

print(f"{len(observations)} total observations" + "\n")
print("Listed target names:")
print(set(observations['target_name']))

Above you can see there are two type of observations we get for Comet-67P, and this query yields a total of 140 observations. The `target_name` criteria is not always the most reliable way to retrieve all desired observations, as it runs into the same pitfalls that `instrument_name` has, where there are multiple names for the same object. In the remainder of this notebook, we will construct a bare-bones ephemeris using the filtered MAST observations of this object and their metadata. We will then do some reverse engineering to query for the target based on coordinates using the ephemeris table, and hope that we get the same results back! Let's begin:

For simplicity, let's work only with the `'COMET-67P-CHURYUMOV-GERASIMENK'` observations to create our ephemeris.

In [ ]:
mask = observations["target_name"] == "COMET-67P-CHURYUMOV-GERASIMENK"
filtered_observations = observations[mask]
filtered_observations

Now that we have our filtered observations, let's sort the rows of this table based on the `t_min` criteria, which refers to the start time of the exposure in MJD.

In [ ]:
filtered_observations.sort("t_min")

Now that we've sorted our table, let's construct a basic ephemeris showing the path of our object over time (with `t_min`, or exposure start in MJD, as our time component):

In [ ]:
ephemeris = Table([filtered_observations["s_ra"],
                   filtered_observations["s_dec"],
                   filtered_observations["t_min"]], names=("ra", "dec", "t_min"))

Let's display the contents of our ephemeris, and use it to generate a plot of the comet's path:

In [ ]:
ephemeris.sort("t_min")
ephemeris.show_in_notebook()

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(ephemeris[0:18]['ra'], ephemeris[0:18]['dec'], color='royalblue', s=200, lw=1., edgecolor='k')

plt.xlabel("Right Ascension (deg)", fontsize=20)
plt.ylabel("Declination (deg)", fontsize=20)

plt.title("Sky Coordinates", fontsize=20)

plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.grid()

Lastly, let's do some reverse engineering and use the RA, Declination, and `t_min` values from the ephemeris to query for observations of the comet once again, and ensure that no observations were missed using the `*` wildcard from the previous criteria query for this example. Be aware that the following cell may take a little longer to run:

In [ ]:
tables = []

for row in ephemeris:
    ra, dec, t_min = row["ra"], row["dec"], row["t_min"]
    coords = f'{ra} {dec}'
    observations = Observations.query_criteria(coordinates=coords,
                                               target_name="COMET-67P-CHURYUMOV-GERASIMENK",
                                               instrument_name="ACS/WFC",
                                               t_min=[t_min-1, t_min+1])

    tables.append(observations)

In [ ]:
ephemeris_observations = unique(vstack(tables), keys="t_min")
ephemeris_observations

In [ ]:
ephemeris_observations.sort("t_min")
filtered_observations.sort("t_min")
ephemeris_observations["t_min"] == filtered_observations["t_min"]

Here we can see that there are no rows in the new `ephemeris_observations` table that don't match the original query results (`filtered_observations`), so the wildcard did catch all observations for Comet 67P.

## Resources

The following is a list of resources that were referenced throughout the tutorial, as well as some additional references that you may find useful:

* https://astroquery.readthedocs.io/en/latest/mast/mast.html#observation-criteria-queries

## Citations

If you use any of astroquery's tools for published research, please cite the authors. Follow this link for more information about citing astroquery:

* [Citing astroquery](https://github.com/astropy/astroquery/blob/main/astroquery/CITATION)

## About This Notebook

If you have comments or questions on this notebook, please contact us through the Archive Help Desk e-mail at archive@stsci.edu. <br>
<br>
Author(s): Jenny V. Medina <br>
Keyword(s): Tutorial, astroquery, wildcards, moving, target <br>
Last Updated: Jun 2023